# Test biaslyze with the toxic comments dataset

Data source: https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('/home/tobias/Repositories/biaslyze/')

In [3]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score

## Load and prepare data

In [4]:
df = pd.read_csv("../data/jigsaw-toxic-comment-classification/train.csv"); df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [5]:
# make the classification problem binary
df["target"] = df[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].sum(axis=1) > 0

## Train a BoW-model

In [6]:
clf = make_pipeline(TfidfVectorizer(min_df=10, max_features=10000, stop_words="english"), LogisticRegression())

In [7]:
clf.fit(df.comment_text, df.target)

/home/tobias/.cache/pypoetry/virtualenvs/biaslyze-OjqpgMw8-py3.10/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('tfidfvectorizer',
                 TfidfVectorizer(max_features=10000, min_df=10,
                                 stop_words='english')),
                ('logisticregression', LogisticRegression())])

In [8]:
train_pred = clf.predict(df.comment_text)
print(accuracy_score(df.target, train_pred))

0.9605755431751384


## Test LIME based bias detection with keywords

In [53]:
from biaslyze.evaluators import LimeBiasEvaluator
from biaslyze.bias_detectors import LimeKeywordBiasDetector

In [64]:
bias_detector = LimeKeywordBiasDetector(
    bias_evaluator=LimeBiasEvaluator(n_lime_samples=1000),
    n_top_keywords=30,
    use_tokenizer=True
)

In [65]:
test_texts = df.comment_text.sample(1000)
detection_res = bias_detector.detect(texts=test_texts, predict_func=clf.predict_proba)

2023-04-14 11:16:08.659 | INFO     | biaslyze.concept_detectors:detect:33 - Started keyword-based concept detection on 5000 texts...
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5000/5000 [01:33<00:00, 53.37it/s]
2023-04-14 11:17:42.356 | INFO     | biaslyze.concept_detectors:detect:49 - Done. Found 1173 texts with protected concepts.
2023-04-14 11:17:42.360 | INFO     | biaslyze.evaluators:evaluate:44 - Started bias detection on 1173 samples...
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1173/1173 [14:21<00:00,  1.36it/s]


In [66]:
detection_res.summary()

Detected 807 samples with potential issues.
    Potentially problematic concepts detected: [('gender', 615), ('nationality', 170), ('religion', 67)]
    Based on keywords: [('he', 215), ('his', 150), ('english', 71), ('her', 53), ('she', 45), ('man', 42), ('guy', 35), ('guys', 29), ('mother', 21), ('son', 20), ('jews', 18), ('german', 17), ('muslim', 16), ('father', 15), ('jewish', 15), ('men', 13), ('chinese', 12), ('israel', 12), ('christian', 11), ('women', 11)].


In [67]:
detection_res.details(group_by_concept=True)

Concept: gender
[{'reason': ['guys'],
  'text': 'hi Dave6\n'
          '\n'
          'ive posted this on the clairvoyance talk page after someone '
          'completely removed the clar#irsentience article which had gone '
          'through a process of months of work , rewrites , endless discussion '
          ', removasl of personal matereal , edits for style and content .... '
          'endless work on correct sourcing and wikpedia referencing and '
          'formatting.....  the article which had been arrived at with co '
          'operation and discussion over many weeks and v man who himself put '
          'some hours into grammer , punctuation ... re editting for stlye and '
          'content ... and much help in correct sourcing and referencing  '
          '.....\n'
          '\n'
          'this article was the sum  of many peoples input , effort and hard '
          'work  so i dont know how or who removed it so thoughtlessly and put '
          'the meaningless redi

In [68]:
test_df = df.iloc[test_texts.index]

In [69]:
test_df[test_df.target].reset_index().comment_text[4]

"Bubble and Squeak?  \n\nThe fact that Bubble and Squeak is is favourite dish is linked to a reference that refers to his divorce. Its clearly fallacious and has been put in as a joke. \n\nIts prolly you bloody Yanks that can't see that its stupid. bubble and squeak weekly? Jesus, give me a break. \n\nI'm great and you all suck. \n\nBye"

In [71]:
from bokeh.io import show, output_notebook

output_notebook()

dashboard = detection_res.dashboard()
show(dashboard)

Loading BokehJS ...

In [74]:
pos_dashboard = detection_res.dashboard(use_position=True)
show(pos_dashboard)

## Testing a sentiment analysis model from huggingface

In [18]:
from transformers import pipeline
from torch.utils.data import Dataset


classifier = pipeline(
    model="distilbert-base-uncased-finetuned-sst-2-english",
    top_k=None,
    padding=True,
    truncation=True
)

In [19]:
class MyDataset(Dataset):
    def __init__(self, data):
        super().__init__()
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, i):
        return self.data[i]


def predict_sentiment(texts):
    data = MyDataset(texts)
    proba = []
    for res in classifier(data):
        proba_array = []
        for p in sorted(res, key=lambda d: d['label'], reverse=True):
            proba_array.append(p.get("score"))
        proba.append(np.array(proba_array))
    return np.array(proba) / np.array(proba).sum(axis=1)[:,None]

In [20]:
bias_detector = LimeKeywordBiasDetector(
    bias_evaluator=LimeBiasEvaluator(n_lime_samples=500),
    n_top_keywords=10,
    use_tokenizer=True
)

In [21]:
test_texts = df.comment_text.sample(50)
detection_res = bias_detector.detect(texts=test_texts, predict_func=predict_sentiment)

2023-04-13 12:04:44.419 | INFO     | biaslyze.concept_detectors:detect:33 - Started keyword-based concept detection on 50 texts...
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:01<00:00, 41.77it/s]
2023-04-13 12:04:45.620 | INFO     | biaslyze.concept_detectors:detect:49 - Done. Found 10 texts with protected concepts.
2023-04-13 12:04:45.621 | INFO     | biaslyze.evaluators:evaluate:42 - Started bias detection on 10 samples...
  0%|                                                                                                                                                                                              | 0/10 [01:02<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
detection_res.summary()

In [ ]:
detection_res.details(group_by_concept=True)

## !! Very Experimental !!: Test masked language model based bias detection with keywords

In [ ]:
from biaslyze.bias_detectors import MaskedKeywordBiasDetector
from biaslyze.concept_detectors import KeywordConceptDetector

In [ ]:
bias_detector = MaskedKeywordBiasDetector(n_resample_keywords=15, use_tokenizer=True)

In [ ]:
detection_res = bias_detector.detect(
    texts=df.comment_text.sample(1000),
    predict_func=predict_sentiment
)

In [ ]:
detection_res.summary()

In [ ]:
detection_res.details()